In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import helper as hlp

# Distance function for GMM
def distanceFunc(X, MU):
    # Inputs
    # X: is an NxD matrix (N observations and D dimensions)
    # MU: is an KxD matrix (K means and D dimensions)
    # Outputs
    # pair_dist: is the squared pairwise distance matrix (NxK)
    K = MU.shape[0]
    N = X.shape[0]
    X = tf.tile(tf.expand_dims(X,1), [1,K,1])
    MU = tf.tile(tf.expand_dims(MU,0), [N,1,1])
    pair_dist = tf.reduce_sum(tf.math.squared_difference(X, MU), 2)
    return pair_dist

def log_GaussPDF(X, mu, sigma):
    # Inputs
    # X: N X D
    # mu: K X D
    # sigma: K X 1

    # Outputs:
    # log Gaussian PDF N X K

    D = tf.to_float(X.shape[1])
    sigma = tf.squeeze(sigma)
    log_term = -D/2 * tf.log(2 * np.pi * sigma)

    X = tf.dtypes.cast(X, tf.float32)
    x_dist = distanceFunc(X, mu)
    exp_term = - x_dist / (2 * sigma)

    return log_term + exp_term

def log_posterior(log_PDF, log_pi):
    # Input
    # log_PDF: log Gaussian PDF N X K
    # log_pi: K X 1

    # Outputs
    # log_post: N X K

    log_pi = tf.squeeze(log_pi)

    sum_term = hlp.reduce_logsumexp(log_PDF + log_pi,keep_dims=True)
    return tf.add(log_pi, log_PDF) - sum_term

"""
need to implement the loss function using log-sum-exp function (in helper file) and perform MLE by directly
optimizing log likelihood function using gradeint descent in Tensorflow
    -> use tf.train.AdamOptimizer(learning_rate=0.1, beta1=0.9, beta2=0.99, epsilon=1e-5) 
       as gradient descent optimizer

-> must place pi through a softmax function first before input into log_GaussPDF function
    -> use logsoftmax helper function
-> must place sigma into constraints with exp(sigma) before input into log_GaussPDF function


"""

#part 2.2

# Loading data
#data = np.load('data100D.npy')
data = np.load('data2D.npy')
[num_pts, dim] = np.shape(data)
is_valid = 1
# For Validation set
if is_valid:
  valid_batch = int(num_pts / 3.0)
  np.random.seed(45689)
  rnd_idx = np.arange(num_pts)
  np.random.shuffle(rnd_idx)
  val_data = data[rnd_idx[:valid_batch]]
  data = data[rnd_idx[valid_batch:]]
  X_validate_ = val_data

else:
    X_validate_ = data

n_iterations = 200
K = 3
D = dim
N_train = np.shape(data)[0]
N_validate = np.shape(val_data)[0]
X_train_ = data
#X_validate_ = data

graph = tf.Graph()
loss = np.zeros((n_iterations,1), dtype=np.float32)
valid_losses = np.zeros((n_iterations,1), dtype=np.float32)

with graph.as_default():
    #X = tf.compat.v1.placeholder(dtype='float32', shape=[N_train, D]) #X for X_train
    X = tf.compat.v1.placeholder(dtype='float32', shape=[None, D]) #X for X_train
    X_validate = tf.compat.v1.placeholder(dtype='float32', shape=[N_validate, D])

    #X_placeholder_train = tf.placeholder("float", [N_train, D], "X")
    #X_placeholder_valid = tf.placeholder("float", [N_validate, D], "X")
    #X = tf.placeholder("float", [None, D], "X")

    MU = tf.Variable(tf.initializers.truncated_normal(mean=0, stddev=1, dtype=tf.dtypes.float32)(shape=(K,D)))

    sigma = tf.Variable(tf.initializers.truncated_normal(mean=0, stddev=1, dtype=tf.dtypes.float32)(shape=(K,1)))
    sigma = tf.exp(sigma)
    pi = tf.Variable(tf.initializers.truncated_normal(mean=0, stddev=1, dtype=tf.dtypes.float32)(shape=(K,1)))
    pi = tf.squeeze(hlp.logsoftmax(pi))


    #log_gauss_pdf_train = log_GaussPDF(X_train_, MU, sigma)
    #log_gauss_pdf_valid = log_GaussPDF(X_validate_, MU, sigma)
    #log_gauss_pdf = log_GaussPDF(X_placeholder_train, MU, sigma)
    log_gauss_pdf = log_GaussPDF(X, MU, sigma)
    


    #P_x_train = pi + log_gauss_pdf_train
    #P_x_valid = pi + log_gauss_pdf_valid

    P_x = pi + log_gauss_pdf

    #now need to calculate training loss
    L_train = -1 * tf.reduce_sum(hlp.reduce_logsumexp(P_x, keep_dims=True))

    #optimizer = tf.train.AdamOptimizer(learning_rate=0.1, beta1=0.9, beta2=0.99, epsilon=1e-5).minimize(loss)
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.1, beta1=0.9, beta2=0.99, epsilon=1e-5).minimize(L_train)

    """
    # Calculate Pair Distance for Validation Set
    pair_dist_validate = distanceFunc(X_validate, MU)
    """
    # Calculate Pair Distance for Training Set
    #pair_dist_train = distanceFunc(X_train, MU)
    pair_dist_train = distanceFunc(X, MU)


    #pred_train = tf.argmax(tf.nn.softmax(log_posterior(log_gauss_pdf_train, pi)), 1)
    #pred_valid = tf.argmax(tf.nn.softmax(log_posterior(log_gauss_pdf_valid, pi)), 1)
    #min_prob = tf.argmax(tf.nn.softmax(log_posterior(log_gauss_pdf, pi)), 1)
    posterior_prob = log_posterior(log_gauss_pdf, pi)
    min_prob_to_grad_descent = tf.argmax(posterior_prob, 1)
    

with tf.Session(graph=graph) as session:
    # Init
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()

    # Train
    for n in range(n_iterations):
        """
        print("X_train.shape", X.shape)
        print("X_train_.shape", X_train_.shape)
        print("X_validate.shape", X_validate.shape)
        print("X_validate_.shape", X_validate_.shape)
        """
        optimizer_, MU_, prediction, loss_ = session.run(
                                                        [optimizer, MU, min_prob_to_grad_descent, L_train],
                                                        feed_dict={X: X_train_})
        
        #optimizer_, MU_, pred_, loss_ = session.run(
        #                                        [optimizer, MU, pred, L_train],
        #                                        feed_dict={X: data})
        loss[n] = loss_
        if(is_valid):
            
            _, _, _,valid_loss  = session.run(
                                            [optimizer, MU, min_prob_to_grad_descent, L_train],
                                            feed_dict={X: X_validate_})
            
            #_, _, _, valid_loss = session.run(
            #                                [optimizer, MU, pred, L_train],
            #                                feed_dict={X: val_data})
            valid_losses[n] = valid_loss


    # Label Data
    #min_dist = np.tile(np.amin(pair_dist_validate_, 1), [K,1]).transpose()
    #y = np.where(pair_dist_validate_ == min_dist)[1]

    print("----------------------------------------")
    print("pi", pi)
    print("pi.eval()", pi.eval())
    #tf.print(pi, [pi])
    print("sigma", sigma)
    print("sigma.eval()", sigma.eval())

    print("final training loss", loss[-1])
    print("final validation loss", valid_losses[-1])
    

    # Plot
    fig = plt.gcf()
    fig.set_size_inches(10, 4)
    plt.subplot(1,2,1)
    plt.plot(np.arange(n_iterations), loss, label="Training Loss")
    plt.plot(np.arange(n_iterations), valid_losses, label="Validation Loss")
    plt.xlabel('Number of Updates')
    plt.ylabel('Training Loss')
    plt.title('Training Curve for KMeans Clustering for K='+str(K))
    plt.legend()

    plt.subplot(1,2,2)


    #prediction=np.int32(prediction)

    for i in range(K):
        plt.scatter(data[:, 0], data[:, 1], c=prediction)#, cmap=plt.get_cmap('Set3'), s=25, alpha=0.6)
    
    #plt.scatter(X_validate_[:, 0], X_validate_[:, 1], c=y)
    #plt.scatter(X_train_[:, 0], X_train_[:, 1], c=y)

    print("Means MU:", MU_)

    plt.scatter(MU_[:, 0], MU_[:, 1], marker='x', c='k')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Clusters on Scatter Plot K='+str(K))

    plt.show()
    fig.savefig('2_1_K_3.png')
















TypeError: ignored